In [ ]:
import tensorflow as tf
from tensorflow import keras as keras
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from numpy.random import randn

In [ ]:
Vox_Lat=np.load('/content/drive/MyDrive/DataSets/Voxel_Lattices.npy')
Vox_Lat_2=np.load('/content/drive/MyDrive/DataSets/Lattice_Voxel_Swapped.npy')

In [ ]:
Lat=np.concatenate((Vox_Lat,Vox_Lat2),axis=0)
x_train=np.expand_dims(Vox_Lat[:5500],4)
x_test=np.expand_dims(Vox_Lat[5500:],4)

In [ ]:
import keras.backend as K

def custom_loss(y_true, y_pred):
    a=K.sum(K.abs(K.log(K.abs(y_true[:,31,0,0,0]))-K.log(K.abs(y_pred[:,31,0,0,0])))*2)
    b=K.sum(K.abs(K.log(K.abs(y_true[:,0,31,0,0]))-K.log(K.abs(y_pred[:,0,31,0,0])))*2)
    c=K.sum(K.abs(K.log(K.abs(y_true[:,0,0,31,0]))-K.log(K.abs(y_pred[:,0,0,31,0])))*2)
    ab=K.sum(K.abs(K.log(K.abs(y_true[:,31,31,0,0]))-K.log(K.abs(y_pred[:,31,31,0,0])))*2)
    bc=K.sum(K.abs(K.log(K.abs(y_true[:,0,31,31,0]))-K.log(K.abs(y_pred[:,0,31,31,0])))*2)
    ac=K.sum(K.abs(K.log(K.abs(y_true[:,31,0,31,0]))-K.log(K.abs(y_pred[:,31,0,31,0])))*2)
    loss1=a+b+c+ab+bc+ac
    loss2 = K.mean(K.abs(y_true - y_pred))

    err=loss1+10*loss2

    return err

In [ ]:
import keras
from keras import layers
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=60, restore_best_weights=True)

input_img = keras.Input(shape=(32, 32, 32, 1))

x = layers.Conv3D(16, 3, strides = (2,2,2), activation='relu', padding='same')(input_img)
x = layers.Conv3D(16, 3,strides = (2,2,2), activation='relu', padding='same')(x)
x = layers.Conv3D(8, 3, strides = (2,2,2),activation='tanh', padding='same')(x)
encoded = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(encoded)
x = layers.Reshape((4,4,4,8))(x)
x = layers.Conv3DTranspose(4, 8, strides=(2,2,2), activation='relu', padding='same')(x)
x = layers.Conv3DTranspose(8, 8, strides=(2,2,2), activation='relu', padding='same')(x)
x = layers.Conv3DTranspose(16, 16, strides=(2,2,2), activation='relu', padding='same')(x)
decoded = layers.Conv3D(1, 3, activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss=custom_loss, metrics=[custom_loss,'mae'])
autoencoder.summary()

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[callback])